In [2]:
# import pandas as pd
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import control as ct

In [4]:
%matplotlib

Using matplotlib backend: <object object at 0x00000125BA1B1A90>


In [9]:
#model response
m = 0.002
R = 0.02
g = 9.8
L = 0.20
d = 0.12
J = (2/3)*m*R**2
s = ct.tf('s')
G = ((m*g*d)/(L*((J/R**2)+m)))*(1/s**2)

Kp = 65
Ki = 162
Kd = 6.5

C = Kp+Ki/s+Kd*s
print(C)
plant = ct.feedback(G*C)
T,U = ct.step_response(-15*plant)
fig8,ax8 = plt.subplots()

L = len(U)
zeros = []
for i in range(0,L-1):
    if abs((U[-1]-U[i-1])/U[-1])>0.05 and abs((U[-1]-U[i+1])/U[-1])<0.05:
        zeros.append(T[i])
for i in range(0,L):
    if (U[-1]-U[i])/U[-1]<0.05 and T[i]>=zeros[-1]:
        I = i
        sett=T[i]
        Usett = U[i]
        break

dict = ct.step_info(15*plant)
print(dict)
over = dict['Overshoot']
print('Overshoot = ',over,'%')
rise = dict['RiseTime']
print('Rise time =',rise,'s')
print("Settling time =",sett,'s')

ax8.plot(T,U+30,label = 'response')
ax8.axhline(15,color = 'red',linestyle ='dotted',label = 'setpoint')
ax8.axhline(U[-1]*0.95+30,color = 'orange',linestyle = 'dotted',label = '5% lower')
ax8.axhline(U[-1]*1.05+30,color = 'green',linestyle = 'dotted',label = '5% upper')
ax8.grid()
ax8.plot(sett,Usett+30,'*',label = "settling point",color = 'r')
#ax8.vlines(0.45,ymin=15,ymax = U[-1],color='r',label = "steady state error")
ax8.vlines(T[np.argmin(U)],ymin=np.min(U)+30,ymax=15,label = 'Overshoot', color = 'red')
ax8.legend()
ax8.set_ylabel('Displacement [cm]')
ax8.set_xlabel('Time [s]')
fig8.suptitle('SIMC Tuned Model Step Response for Ping Pong Ball')


6.5 s^2 + 65 s + 162
--------------------
         s

{'RiseTime': 0.05696695116220753, 'SettlingTime': 0.6123947249937309, 'SettlingMin': 14.586155531354137, 'SettlingMax': 18.52370091880247, 'Overshoot': 23.49133945868312, 'Undershoot': 0, 'Peak': 18.52370091880247, 'PeakTime': 0.1424173779055188, 'SteadyStateValue': 15.0}
Overshoot =  23.49133945868312 %
Rise time = 0.05696695116220753 s
Settling time = 0.3133182313921414 s


Text(0.5, 0.98, 'SIMC Tuned Model Step Response for Ping Pong Ball')

In [19]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(5,1,figsize=(6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figraw,axsraw = plt.subplots(5,1,figsize=(6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figz,axz = plt.subplots(5,1,figsize = (6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)

fig.suptitle('Individual SIMC Tuned Responses for Ping Pong Ball')
figraw.suptitle('Unfiltered SIMC Tuned Responses for Ping Pong Ball')
figz.suptitle('Motor Angle for SIMC Tuned Ping Pong Ball Response')

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of SIMC Tuned Ping Pong Ball Responses')
#fig3.suptitle('Unfiltered Average of Manually Tuned Ping Pong Ball Responses')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Distance [cm]')
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 1000, 'lp', fs=50000, output='sos')

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1420
for i in range(1,6):
    df = pd.read_csv('PING_SIMC_{}.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs[i-1].plot(timenew,filtered,label = 'filtered',color = 'b')
    axs[i-1].plot(timenew,distance,alpha=0.4,label = 'unfiltered',color = 'darkslategrey')
    axs[i-1].set_xlabel('Time [s]')
    axs[i-1].set_ylabel('Distance [cm]')
    axs[i-1].axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
    axs[i-1].grid()
    
    axsraw[i-1].plot(timenew,distance)
    axsraw[i-1].set_xlabel('Time [s]')
    axsraw[i-1].set_ylabel('Distance [cm]')
    axsraw[i-1].grid()
    
    axz[i-1].plot(timenew,angle_filt,label = 'filtered',color = 'b')
    axz[i-1].plot(timenew,angle,alpha=0.35,label = 'unfiltered',color = 'darkslategrey')
    axz[i-1].set_xlabel('Time [s]')
    axz[i-1].set_ylabel('Servo Angle [degrees]')
    axz[i-1].grid()
    axz[i-1].axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()
    
axs[0].legend(loc='lower right')
axz[0].legend(loc = 'lower right')
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<1 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ticks = np.linspace(0,30,31)
ax2.set_yticks(ticks)
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')

#ax3.plot(time_ave,dist_ave,label = 'response')

1420
1473
1443
1657
1711
[0.015, 0.034600000000000006, 0.0528, 0.0718, 0.09159999999999999, 0.1096, 0.12940000000000002, 0.1474, 0.16720000000000002, 0.18539999999999995, 0.205, 0.22360000000000002, 0.24219999999999997, 0.262, 0.2800000000000001, 0.29979999999999996, 0.31799999999999995, 0.3376, 0.3558, 0.3754, 0.3942, 0.41259999999999997, 0.4324, 0.4504, 0.4702, 0.4888, 0.508, 0.5267999999999999, 0.5458000000000001, 0.5648, 0.5831999999999999, 0.603, 0.6212, 0.641, 0.6592, 0.6786000000000001, 0.6971999999999999, 0.7165999999999999, 0.7350000000000001, 0.7536000000000002, 0.7726, 0.7914000000000001, 0.8101999999999998, 0.829, 0.8484, 0.867, 0.8859999999999999, 0.9046, 0.9228, 0.9422, 0.9602, 0.9793999999999998, 0.9978, 1.0168, 1.0352000000000001, 1.0535999999999999, 1.0728000000000002, 1.0908, 1.1096, 1.1278000000000001, 1.1466, 1.165, 1.1832, 1.2016, 1.2198, 1.2384, 1.2567999999999997, 1.2753999999999999, 1.2932, 1.3115999999999999, 1.33, 1.3476, 1.3662, 1.3843999999999999, 1.40319999

In [6]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)
figraw,axsraw = plt.subplots()

figz,axz = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)

fig.suptitle('Individual SIMC Tuned Responses for Ping Pong Ball')
figraw.suptitle('Unfiltered SIMC Tuned Responses for Ping Pong Ball')
figz.suptitle('Motor Angle for SIMC Tuned Ping Pong Ball Response')

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of SIMC Tuned Ping Pong Ball Responses')
#fig3.suptitle('Unfiltered Average of Manually Tuned Ping Pong Ball Responses')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Distance [cm]')
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 1000, 'lp', fs=50000, output='sos')
lines = ['solid','dashed','dotted','dashdot','solid']

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1420
ticksang = np.linspace(-30,30,11)
for i in range(1,6):
    df = pd.read_csv('PING_SIMC_{}.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs.plot(timenew,filtered,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axs.set_xlabel('Time [s]')
    axs.set_ylabel('Distance [cm]')
    
    axs.grid()
    
    axsraw.plot(timenew,distance)
    axsraw.set_xlabel('Time [s]')
    axsraw.set_ylabel('Distance [cm]')
    axsraw.grid()
    
    axz.plot(timenew,angle_filt,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axz.set_xlabel('Time [s]')
    axz.set_ylabel('Servo Angle [degrees]')
    axz.grid()

    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()

    
    
    
axs.axhline(15,color = 'r',linestyle = 'dotted',label = 'Setpoint')    
axz.axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
axs.legend(loc = 'lower right')
axz.legend(loc = 'lower right')
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<1 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')
ticks = np.linspace(0,25,26)
ax2.set_yticks(ticks)

#ax3.plot(time_ave,dist_ave,label = 'response')

1420
1473
1443
1657
1711
[0.015, 0.034600000000000006, 0.0528, 0.0718, 0.09159999999999999, 0.1096, 0.12940000000000002, 0.1474, 0.16720000000000002, 0.18539999999999995, 0.205, 0.22360000000000002, 0.24219999999999997, 0.262, 0.2800000000000001, 0.29979999999999996, 0.31799999999999995, 0.3376, 0.3558, 0.3754, 0.3942, 0.41259999999999997, 0.4324, 0.4504, 0.4702, 0.4888, 0.508, 0.5267999999999999, 0.5458000000000001, 0.5648, 0.5831999999999999, 0.603, 0.6212, 0.641, 0.6592, 0.6786000000000001, 0.6971999999999999, 0.7165999999999999, 0.7350000000000001, 0.7536000000000002, 0.7726, 0.7914000000000001, 0.8101999999999998, 0.829, 0.8484, 0.867, 0.8859999999999999, 0.9046, 0.9228, 0.9422, 0.9602, 0.9793999999999998, 0.9978, 1.0168, 1.0352000000000001, 1.0535999999999999, 1.0728000000000002, 1.0908, 1.1096, 1.1278000000000001, 1.1466, 1.165, 1.1832, 1.2016, 1.2198, 1.2384, 1.2567999999999997, 1.2753999999999999, 1.2932, 1.3115999999999999, 1.33, 1.3476, 1.3662, 1.3843999999999999, 1.40319999